In [167]:
import pandas as pd
import numpy as np
import scipy
import sys, os, pickle
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential

#classifier models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import SVC


#metrics
from sklearn.metrics import accuracy_score, log_loss, roc_curve, auc

In [168]:
df = pd.read_csv(r'./data/features_3_sec.csv')
df = df.drop(labels='filename', axis=1)

In [172]:
y = LabelEncoder().fit_transform(df.iloc[:,-1])
X = StandardScaler().fit_transform(np.array(df.iloc[:,:-1], dtype=float))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

## Deep Learning model

In [ ]:
def trainModel(model, epochs, optimizer):
    batch_size = 128
    #callback = myCallback()
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    return model.fit(X_train, y_train, validation_data=(X_test, y_test),
                     epochs=epochs, batch_size=batch_size)

def plotValidate(history):
    print("Validation Accuracy", max(history.history['val_accuracy']))
    pd.DataFrame(history.history).plot(figsize=(12,6))
    plt.show()

In [ ]:
units, rate = 64, 0.2
model = keras.models.Sequential([
    keras.layers.Dense(units*2**3, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(rate),
    
    keras.layers.Dense(units*2**2, activation='relu'),
    keras.layers.Dropout(rate),
    
    keras.layers.Dense(units*2**1, activation='relu'),
    keras.layers.Dropout(rate),
    
    keras.layers.Dense(units, activation='relu'),
    keras.layers.Dropout(rate),
    
    keras.layers.Dense(10, activation='softmax'),
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [166]:
model.summary()

AttributeError: 'AdaBoostClassifier' object has no attribute 'summary'

In [ ]:
model_history = trainModel(model=model, epochs=600, optimizer='adam')

Train on 6693 samples, validate on 3297 samples
Epoch 1/600
6693/6693 [==============================] - 2s 347us/sample - loss: 1.7018 - acc: 0.4009 - val_loss: 1.1784 - val_acc: 0.5866
Epoch 2/600
6693/6693 [==============================] - 1s 90us/sample - loss: 1.1701 - acc: 0.5933 - val_loss: 0.8748 - val_acc: 0.7040
Epoch 3/600
6693/6693 [==============================] - 1s 79us/sample - loss: 0.9305 - acc: 0.6834 - val_loss: 0.7721 - val_acc: 0.7398
Epoch 4/600
6693/6693 [==============================] - 1s 78us/sample - loss: 0.8026 - acc: 0.7282 - val_loss: 0.7081 - val_acc: 0.7513
Epoch 5/600
6693/6693 [==============================] - 1s 97us/sample - loss: 0.7138 - acc: 0.7606 - val_loss: 0.6387 - val_acc: 0.7904
Epoch 6/600
6693/6693 [==============================] - 1s 98us/sample - loss: 0.6324 - acc: 0.7880 - val_loss: 0.5700 - val_acc: 0.8150
Epoch 7/600
6693/6693 [==============================] - 1s 97us/sample - loss: 0.5869 - acc: 0.8050 - val_loss: 0.5676 - v

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=128)

3297/3297 [==============================] - 0s 29us/sample - loss: 0.5206 - acc: 0.9254


In [ ]:
print('The test loss is: ', test_loss)
print('The Best test Accuracy is: ', test_acc*100)

The test loss is:  0.5205980658187699
The Best test Accuracy is:  92.53867268562317


# Other ML models

In [43]:
categorylist = list(df.select_dtypes(['object']))
tempdf = pd.get_dummies(df, columns=categorylist)
label_columns = [x for x in tempdf.columns if x.startswith('label')]
X = tempdf.drop(label_columns, axis=1)
y = tempdf[label_columns].copy()

In [122]:
encoder = LabelEncoder()
categorylist = list(df.select_dtypes(['object']))
y = df[categorylist]
y_encoded = encoder.fit_transform(y)

D:\Sam\miniconda3\envs\venv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [47]:
class OldDataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [148]:
def from_multilabel_to_onehot(array, column='label', return_columns=False):
    df = pd.DataFrame(y_test, columns=[column])
    df = pd.get_dummies(df, columns=[column])
    
    if return_columns:
        return [x.replace('label_','') for x in df.columns]
    return df.to_numpy()

In [48]:
num_attribs = list(X.select_dtypes(['int64', 'float64']))
pipeline = Pipeline([
    ('selector', OldDataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

In [49]:
X_prepared = pipeline.fit_transform(X)
# y_prepared = y.to_numpy()[:,1]
y_prepared = y.to_numpy()

In [124]:
sss = StratifiedShuffleSplit(n_splits=10, test_size= 0.33, random_state=0)

classifiers = [
    KNeighborsClassifier(),
    ExtraTreeClassifier(),
    DecisionTreeClassifier(),
]

In [149]:
res = []
labels = [x.replace('label_','') for x in y.columns]
for train_idx, test_idx in sss.split(X_prepared, y_prepared):
    X_train, X_test = X_prepared[train_idx], X_prepared[test_idx]
    y_train, y_test = y_prepared[train_idx], y_prepared[test_idx]
    for clf in classifiers:
        name = clf.__class__.__name__
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_probas = clf.predict_proba(X_test)
        acc = accuracy_score(y_test, y_pred)
        loss = log_loss(y_test, y_pred)
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(y_test[:,i], y_pred_probas[i][:,1])
            roc_auc[label] = auc(fpr[label], tpr[label])
            res.append([name, label, acc, loss, roc_auc[label]])

In [159]:
res_svc = []
clf = SVC(probability=True)
for fold_idx, (train_idx, test_idx) in enumerate(sss.split(X_prepared, y_encoded)):
    print(fold_idx)
    X_train, X_test = X_prepared[train_idx], X_prepared[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
    name = clf.__class__.__name__
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_probas = clf.predict_proba(X_test)
    acc = accuracy_score(y_test, y_pred)
    loss = log_loss(y_test, y_pred_probas)
    
    labels = from_multilabel_to_onehot(y_test, return_columns=True)
    y_test = from_multilabel_to_onehot(y_test)
    # y_pred_probas = from_multilabel_to_onehot(y_pred_probas)
    fpr, tpr, roc_auc = dict(), dict(), dict()
    for i, label in enumerate(labels):
        fpr[label], tpr[label], _ = roc_curve(y_test[:,i], y_pred_probas[:,i])
        roc_auc[label] = auc(fpr[label], tpr[label])
        res_svc.append([name, label, acc, loss, roc_auc[label]])

0
1
2
3
4
5
6
7
8
9


In [165]:
log_svc = pd.DataFrame(res_svc, columns=['Classifier','Label', 'Accuracy','Log Loss','AUC'])
log = pd.DataFrame(res, columns=['Classifier','Label', 'Accuracy','Log Loss','AUC'])
a = log_svc.groupby(['Classifier'], as_index=False).mean()
b = log.groupby(['Classifier'], as_index=False).mean()
pd.concat([b,a],axis=0).sort_values(by='Accuracy', ascending=False)

,Classifier,Accuracy,Log Loss,AUC
0,SVC,0.848286,0.457128,0.986214
2,KNeighborsClassifier,0.833910,2.898170,0.986842
0,DecisionTreeClassifier,0.626630,12.875201,0.772335
1,ExtraTreeClassifier,0.529148,16.248974,0.714333
